# Исследование надежности заемщиков


## Откройте таблицу и изучите общую информацию о данных

Импортируем библиотеку и сохраним датафрейм в переменную

In [ ]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

Выведем первые 20 строчек датафрейма `data` на экран.

In [ ]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца. 

In [ ]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Заполним пропуски в столбце total_income медианой

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

Заменим все отрицательные значения положительными с помощью метода.

In [ ]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Однако мы оставим их.

Выведем перечень уникальных значений столбца `children`.

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Выведем еще раз перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Проверим количество пропущенных значений.

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. 

In [ ]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. И удалим их, если присутствуют.

In [ ]:
data.duplicated().sum()

71

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем перечень уникальных целей взятия кредита из столбца `purpose`.

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.


In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:

#функция для разделения на категории по количеству детей
def categorize_children(children):
    try:
        if children <= 0 :
            return 'Бездетный'
        elif children <= 2:
            return 'Малодетный'       
        return 'Многодетный'        
    except:
        return 'нет категории'

In [ ]:
#добавление нового столбца с категориями
data['children_categorize'] = data['children'].apply(categorize_children)

In [ ]:
#сводная таблица с данными по к-ву детей и возвратом кредита в срок
child_categorize = data.pivot_table(index='children_categorize', values='debt', aggfunc=['count','sum','mean']) 
child_categorize.columns=['Количество', 'Сумма', 'Доля просрочки']
display(child_categorize)


,Количество,Сумма,Доля просрочки
children_categorize,,,
Бездетный,14091,1063,0.075438
Малодетный,6860,638,0.093003
Многодетный,380,31,0.081579


**Вывод:** 

Можно сделать вывод, что есть небольшая зависимость между количеством детей и и возвратом кредита в срок. Люди, не имеющие детей, наиболее часто возвращают кредиты в срок. Однако малодетные наиболее часто просрочивают кредиты в сравнении с многодетными. Это выглядит неоднозначно, возможно, нужно иметь больше данных для этой зависимости.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:

#сводная таблица с даными по семейному положению и возвратом кредита в срок 
data_pivot = data.pivot_table(index='family_status', values='debt', aggfunc=['count','sum','mean']) 
data_pivot.columns=['Количество', 'Сумма', 'Доля просрочки']
display(data_pivot)

,Количество,Сумма,Доля просрочки
family_status,,,
Не женат / не замужем,2796,273,0.097639
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246
гражданский брак,4134,385,0.093130
женат / замужем,12261,927,0.075606


**Вывод:** 

Исходя из результатов сводной таблицы, можно сделать вывод, что зависимость есть. Люди, которые проживают в гражданском браке или никогда не были в браке, имеют наиболее высокий процент невозврата кредита в срок. Вдовцы/вдовы в свою очередь наиболее часто возвращают кредиты в срок. 

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:

#Сводная таблица с категориями по уровням дохода и возвратом кредита в срок
income_category = data.pivot_table(index='total_income_category', values='debt', aggfunc=['count','sum','mean'])
income_category.columns=['Количество', 'Сумма', 'Доля просрочки']
print(income_category)

                       Количество  Сумма  Доля просрочки
total_income_category                                   
A                              25      2        0.080000
B                            5014    354        0.070602
C                           15921   1353        0.084982
D                             349     21        0.060172
E                              22      2        0.090909


**Вывод:** 

Исходя из данных сводной таблицы, можно сделать вывод, что наиболее часто в срок возвращают кредиты люди с уровенем дохода 'D', т.е. от 30 001 до 50 000. Задерживать же выплату наиболее склонны люди с уровнем дохода 'E', т.е. до 30 000. Однако их общее количество в таблице по сравнению с другими уровнями дохода меньше, поэтому говорить об определенной зависимости нельзя. 

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [ ]:

purpose_category = data.pivot_table(index='purpose_category', values='debt', aggfunc=['count','sum','mean'])
purpose_category.columns=['Количество', 'Сумма', 'Доля просрочки']
print(purpose_category)

                          Количество  Сумма  Доля просрочки
purpose_category                                           
операции с автомобилем          4279    400        0.093480
операции с недвижимостью       10751    780        0.072551
получение образования           3988    369        0.092528
проведение свадьбы              2313    183        0.079118


**Вывод:** 

Исходя из результатов таблицы, можно сделать вывод, что люди, у которых цель кредита - недвижимость, наиболее часто возвращают кредиты в срок. А также люди, берущие кредит на свадьбу, также склонны возвращать кредит во время. 

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* 

Возможно, пользователь или человек, который предоставлял информацию, не указал о себе какие-либо данные. Или же данные могли повредиться при выгрузке. Если же данные собирались автоматически, также может быть сбой в этой системе. 

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

Ответ: Если в данных присутствуют значения, которые выделяются из общей массы или значения во все пропущены, то с помощью медианы можно нивелировать их влияние. Тогда при  дальнейшем анализе мы получим более объективную информацию.

### Шаг 4: общий вывод.

Таким образом, можно сделать вывод, что количество детей влияет на срок оплаты. У людей, не имеющих детей, наименее вероятна задержка платежей. Семейное положение также влияет на вероятность платежей по кредиту в срок. Вдовцы/вдовы и разведенные в наиболее часто возвращают кредиты в срок, в отличие от людей, которые проживают гражданским браком или никогда не были в браке. Они в свою очередь склонны задерживать платежи. 
В срок также более вероятно вернут кредит взятый на недвижимость и свадьбу, а кредиты на автомобили и образование наиболее склонны возвращать не в срок. 
Однако зависимости между уровнями дохода и возвращением кредитов в срок не было обнаружено.
Можно сказать что для банка наиболее удачным клиентом будет являться: Разведенный человек, не имеющий детей или вдовец/вдова без детей. Наиболее рискованным клиентом в свою очередь является: Человек, проживающий гражданским браком, с детьми или же человек с детьми, который никогда не был в браке. 